# **Used Car Price Predictor Notebook**

### import required libraries

In [60]:
import keras 
import pandas as pd
import pandasql as sqldf
from geopy.geocoders import Nominatim
import time
from zip_to_state import zip_to_state_mapping
import plotly.express as px

## **Phase 1: Data Collection**
#### **Step 1: Extract Training Data from Parquet File**

In [4]:
# time to run: 4m 57s
df = pd.read_csv('used_cars_data.csv')

C:\Users\Daymo\AppData\Local\Temp\ipykernel_31200\2680727636.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('used_cars_data.csv')


# ---------------------------------------------------------------------------

#### **Step 2: Preview the Dataset's Schema**

In [7]:
pd.reset_option('all')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(3)
# df.shape
# df.columns


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,1300.0,I4,Solar Yellow,NaN,NaN,True,Jeep,41.2 in,12.7 gal,Gasoline,NaN,66.5 in,NaN,177.0,Black,NaN,NaN,NaN,True,NaN,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.cargurus.com/images/forsale/202...,['Quick Order Package'],Jeep,5 seats,7.0,Renegade,NaN,"177 hp @ 5,750 RPM",23141.0,NaN,0,2.8,370599.0,Flagship Chrysler,NaN,"200 lb-ft @ 1,750 RPM",A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,2000.0,I4,Narvik Black,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,Black (Ebony),NaN,NaN,NaN,True,NaN,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,8.0,Discovery Sport,NaN,"246 hp @ 5,500 RPM",46500.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,1233,969,NaN,H4,2500.0,H4,NaN,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,NaN,False,NaN,NaN,False,NaN,18.3467,180.9 in,2017-04-25,UNKNOWN,173473508,-66.1098,NaN,"['Alloy Wheels', 'Bluetooth', 'Backup Camera',...",Subaru,5 seats,NaN,WRX STI,3.0,"305 hp @ 6,000 RPM",46995.0,False,0,NaN,370467.0,FIAT de San Juan,False,"290 lb-ft @ 4,000 RPM",M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016


# ---------------------------------------------------------------------------

## **MLOPs Phase 2: Data Ingestion**
#### For the purposes of this tutorial, we will not be ingesting data. However, in an organizational setting, you'll use an ingestion framework, such as Apache Airflow, Apache Spark, or Apache Kafka, to collect data into a Data Lake to create a curated Training Dataset.

## **MLOPs Phase 3: Data Storage **
#### Since we already extracted this data was previously extracted from BigQuery, this phase has already been completed. Typically, you'll use a Cloud Storage Bucket (e.g., AWS S3, Google Cloud Storage) to store all data to be used in ML applications.

# ---------------------------------------------------------------------------

## **MLOps Phase 5: Exploratory Data Analysis (EDA)**

In [9]:

numerical_columns = [ 
       'daysonmarket','listed_date', 'listing_color', 'mileage', 'owner_count', 'price','year'] 

summary_statistics = df[numerical_columns].describe()
rounded_summary_statistics = summary_statistics.round(2)  
print(rounded_summary_statistics)


# categorical_columns = ['body_type','exterior_color','has_accidents','is_new','salvage','trim_name','wheel_system','make_name','model_name', 'frame_damaged','fleet']
# unique_value_counts = df[categorical_columns].nunique()

# value_frequencies = {}
# for col in categorical_columns:
#     value_frequencies[col] = df[col].value_counts()

# print("Count of Unique Values:")
# print(unique_value_counts)

# print("\nFrequency of Each Value:")
# for col, frequencies in value_frequencies.items():
#     print(f"\n{col}:")
#     print(frequencies)

       daysonmarket      mileage  owner_count       price        year
count    3000040.00   2855653.00   1483027.00  3000040.00  3000040.00
mean          76.06     31146.90         1.53    29933.37     2017.73
std          108.88     74586.75         0.92    19566.17        4.18
min            0.00         0.00         1.00      165.00     1915.00
25%           14.00         6.00         1.00    18451.00     2017.00
50%           35.00      8267.00         1.00    26477.00     2020.00
75%           82.00     43662.00         2.00    38220.00     2020.00
max         3599.00  99999988.00        19.00  3299995.00     2021.00


In [69]:

x_range_price = [0, 200000]
x_range_year = [1995, 2022]
x_range_mileage = [0, 100000]

# Create a histogram for 'price' with the specified x-axis range
# fig_price = px.histogram(df, x='price', title='Price Distribution - Outliers removed', range_x=x_range_price,  nbins=500)

# fig_mileage = px.histogram(df, x='mileage', title='Mileage Distribution - Outliers removed', range_x=x_range_mileage,  nbins=32000)

# fig_year = px.histogram(df, x='year', title='Year Distribution - Outliers removed', range_x=x_range_year)

pysqldf = sqldf.sqldf("SELECT dealer_zip, COUNT(*) AS num_listings FROM df GROUP BY dealer_zip")
pysqldf['state'] = pysqldf['dealer_zip'].apply(lambda x: next((state for (zip_range, state) in zip_to_state_mapping.items() if int(str(x)[:5]) >= int(str(zip_range[0])[:5]) and int(str(x)[:5]) <= int(str(zip_range[1])[:5])), 'Unknown'))
max_value = pysqldf['num_listings'].max()
fig_map = px.choropleth(
    pysqldf,
    locations='state',        # Zip code column
    locationmode='USA-states',        # Specify that you're using zip codes
    color='num_listings',         # Data to visualize by zip code
    color_continuous_scale='greens',
    range_color=(0, max_value),  # Use the calculated max_value
    scope='usa'                    # Set the map scope to the USA
)
fig_map.update_geos(fitbounds='locations', visible=False)
fig_map.show()


# fig_price.show()
# fig_mileage.show()
# fig_year.show()


In [ ]:

query = """
    SELECT make_name , model_name, year, price, mileage as count 
    FROM df_cleaned
    WHERE mileage > 999999"""
result_df = ps.sqldf(query)
print(result_df)

In [ ]:
# Count rows with NaN as 'mileage'
nan_mileage_count = df['mileage'].isna().sum()

# Print the count
print("Number of rows with NaN as 'mileage':", nan_mileage_count)
df.shape

#### **Step 1: Create Histograms for Continuous, Numerical Features**

#### **Step 2: Create Correlation Matrix**

#### **Step 3: Show Descriptive Statistics for All Numeric Features**

#### **Step 4: Create Bar Charts for Categorical or Discrete Features**

## **DataPipline (Preparation and Wrangling)**

# ---------------------------------------------------------------------------
#### **Step 1: Use Intuition to Remove Irrelevant Features**

In [ ]:
columns_to_be_removed = ["latitude","longitude","city","vin","back_legroom","bed","bed_height","bed_length","body_type","cabin","city_fuel_economy","combine_fuel_economy","daysonmarket","description","engine_cylinders","engine_displacement","engine_type","franchise_dealer","franchise_make","front_legroom","fuel_tank_volume","fuel_type","height","highway_fuel_economy","horsepower","interior_color","isCab","is_certified","is_cpo","is_oemcpo","length","listed_date","listing_id","main_picture_url","maximum_seating","power","savings_amount","seller_rating","sp_id","sp_name","theft_title","torque","transmission","transmission_display","vehicle_damage_category","wheel_system_display","wheelbase","width".'trimId', 'major_options']
df_cleaned = df.drop(columns=columns_to_be_removed)

#### **Remove outlier from major columns**

#### **Convert all data in each column to target type**

In [ ]:

data_types = {
    'fleet': bool,
    'frame_damaged': bool,
    'has_accidents': bool,
    'is_new': bool,
    'mileage': int,
    'owner_count': int,
    'price': int,
    'salvage': bool,
    'year': int
}

# Use the astype method to convert columns to the specified data types
df = df.astype(data_types)

# If you want to convert 'listing_color', 'make_name', 'model_name',  'trim_name', and 'wheel_system' columns to string
string_columns = ['listing_color', 'make_name', 'model_name', 'trim_name', 'wheel_system']
df[string_columns] = df[string_columns].astype(str)


#### **get rid of redundant year values in trim name col**

In [ ]:

df_cleaned['trim_name'] = df_cleaned['trim_name'].str.replace(r'20\d{2}\.5 ', '', regex=True)

#### **Simple data imputation to fill all the missing mileage values to the average mileage of vehicles of theat same year**

In [ ]:
# Calculate the average mileage for each year
average_mileage_by_year = df.groupby('year')['mileage'].transform('mean')

# Fill NaN values in 'mileage' with the corresponding average for the year
df['mileage'].fillna(average_mileage_by_year, inplace=True)

# Print the updated DataFrame
print(df)

# ---------------------------------------------------------------------------

## **MLOps Phase 6: Feature Engineering**

#### turn the zip code into a US state field and remove all NONE rows**

In [ ]:


pd.options.mode.chained_assignment = None

df_cleaned['state'] = 'Not Found'
# Turn the zipCode Column into a new state column
for index, row in df_cleaned.iterrows():
    zip_code = int(str(row['dealer_zip'])[:5]) 
    for (start, end), state_name in zip_to_state_mapping.items():
        if start <= zip_code <= end:
            df_cleaned.loc[index, 'state'] = state_name
            break

# Remove NULL or Unknown states
undesired_values = ['Not Found']
mask = ~df_cleaned['state'].isin(undesired_values)
df_cleaned = df_cleaned[mask]
df_cleaned = df_cleaned.reset_index(drop=True)

unique_values = df_cleaned['state'].unique()
print(len(unique_values))

In [ ]:
df_cleaned = df_cleaned.drop(columns=['dealer_zip'])

#### **Step 3: Create DataFrame that Merges Pre-Preprocessed Data and Preprocessed Data**

# ---------------------------------------------------------------------------

## **MLOps Phase 7: Model Development**

#### **Step 1: Identify Features and Label**

#### **Step 2: Split Training Data into Training and Evaluation Subsets**

#### **Step 3: Fit Training Subset to Model**

#### **Step 4: Show Feature Importance Metrics**